In [1]:
# !pip3 install beautifulsoup4 -U
import requests
from bs4 import BeautifulSoup
import csv
import urllib.request
from urllib.error import HTTPError

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import os
import pandas as pd
from tqdm import tqdm

###test

## 필요한 데이터 수집
- 옷 이미지 url, 이미지 저장되는 이름, 옷 카테고리, 카테고리 사이즈,개별 옷 url, 그 옷의 소재, 제조자(수입자) 를 딕셔너리로 저장한다.

In [4]:
clothes_dic = {"img_url":None , "img_name":None, "clothes_category":None,
               "brand":None, "category_size":None, "self_clothes_url":None, "clothes_material":None, "clothes_importer":None, "price":None}
clothes_dic

{'img_url': None,
 'img_name': None,
 'clothes_category': None,
 'brand': None,
 'category_size': None,
 'self_clothes_url': None,
 'clothes_material': None,
 'clothes_importer': None,
 'price': None}

In [14]:
#티셔츠만 수집함. #이미지 저장은 안함. 001003004
# https://www.wconcept.co.kr/Men/006001?ccd=3_006001002
large_category_code = {"럭셔리어패럴": "006001"}
small_category_code = { "바지": "006001006"} # "반팔":"006001002", "긴팔": "006001003", "팬츠": "006001006" # "모자": "006004003"
page = 17
code = '006001006'

for cate in tqdm(small_category_code.items()):
    clothes_info_list = []
    for num in range(1, page+1):
        url = f"https://www.wconcept.co.kr/Men/{large_category_code['럭셔리어패럴']}?ccd=3_{code}&page={num}" 
        request_result = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        html = request_result.text
        soup = BeautifulSoup(html, "html.parser")

        raw_imgs = soup.find_all("img")
        img_urls = [img["src"] for img in raw_imgs]
        img_urls = ["https:"+img for img in img_urls if img[2:9]=="product" and "thumbnail" not in img]

        if len(img_urls) < 1:
            break #해당페이지가 없는 경우

        # print(code, num, len(img_urls), img_urls)

        for i, img_url in enumerate(img_urls):
            clothes_dic = {"img_url":None , "img_name":None, "clothes_category":None,
               "brand":None, "category_size":None, "self_clothes_url":None, "clothes_material":None, "clothes_importer":None, "price":None}


            clothes_code = img_url.split("/")[-1]
            clothes_img = f"{large_category_code['럭셔리어패럴']}-{code}-{clothes_code}"

            clothes_dic["img_url"] = img_url
            clothes_dic["img_name"] = clothes_img
            clothes_dic["clothes_category"] = code # code 
            clothes_dic["category_size"] = "small"

            product_code = clothes_code.split("_")[0]
            if ".jpg" in product_code:
                product_code = product_code[:-4] 
            small_clothes_url = f"https://www.wconcept.co.kr/Product/{product_code}"
            clothes_dic["self_clothes_url"] = small_clothes_url

            #여기에서 정보 수집 이뤄져야 함.
            request_result = requests.get(small_clothes_url, headers={"User-Agent": "Mozilla/5.0"})
            html = request_result.text
            soup = BeautifulSoup(html, "html.parser")

            price_selector = "dd > em"
            price = soup.select(price_selector)
            if price:
                clothes_dic["price"] = price[0].text.replace(",","") #가격에 붙은 쉼표 없앰

            clothes_material_selector = "#container > div > div > div.pdt_contents.detail > div.noti_prod_info.st_default > table > tbody > tr:nth-child(1) > td"
            clothes_material = soup.select(clothes_material_selector)
            materials = [material.text for material in clothes_material]
            if materials:
                clothes_dic["clothes_material"] = materials[0]

            importer_selector = "#container > div > div > div.pdt_contents.detail > div.noti_prod_info.st_default > table > tbody > tr:nth-child(4) > td"
            clothes_importer = soup.select(importer_selector)
            importer = [importer.text for importer in clothes_importer]
            if importer:
                clothes_dic["clothes_importer"] = importer[0]


            brand = soup.find_all("h2")
            if brand:
                brand = soup.find_all("h2")[0].text
                brand = brand.replace("\n", "")
                clothes_dic["brand"] = brand

            # print("저장됨", i, clothes_dic)
            clothes_info_list.append(clothes_dic)

    df = pd.DataFrame(clothes_info_list)
    # df.to_csv(f"/content/drive/MyDrive/Colab Notebooks/clothes_info_{code}_page{page}_20230627.csv") # code
        # print("리스트길이",len(clothes_info_list))
        # print(clothes_info_list)

100% 1/1 [09:27<00:00, 567.93s/it]


In [15]:
df5 = df.dropna(axis=0)
df5

,img_url,img_name,clothes_category,brand,category_size,self_clothes_url,clothes_material,clothes_importer,price
0,https://product-image.wconcept.co.kr/productim...,006001-006001006-303117076_GG20620.jpg,006001006,OUR LEGACY,small,https://www.wconcept.co.kr/Product/303117076,100% POLYAMIDE,아워 레가시 / 병행수입,366000
1,https://product-image.wconcept.co.kr/productim...,006001-006001006-303117295_HJ41564.jpg,006001006,MOSCHINO,small,https://www.wconcept.co.kr/Product/303117295,"COTTON 40%, POLYESTER 60%",MOSCHINO,602000
2,https://product-image.wconcept.co.kr/productim...,006001-006001006-303117293_IG16967.jpg,006001006,MOSCHINO,small,https://www.wconcept.co.kr/Product/303117293,COTTON 100%,MOSCHINO,724000
3,https://product-image.wconcept.co.kr/productim...,006001-006001006-303115638_HL14618.jpg,006001006,JIL SANDER,small,https://www.wconcept.co.kr/Product/303115638,상세설명 참조,JIL SANDER,2311000
4,https://product-image.wconcept.co.kr/productim...,006001-006001006-303115217_GN57378.jpg,006001006,Palm Angels,small,https://www.wconcept.co.kr/Product/303115217,80% cotton 20% polyester Lining 100% cotton,PALM ANGELS,704000
...,...,...,...,...,...,...,...,...,...
1525,https://product-image.wconcept.co.kr/productim...,006001-006001006-303084830_GG16447.jpg,006001006,CASABLANCA,small,https://www.wconcept.co.kr/Product/303084830,100% POLYESTER,CASABLANCA,1047000
1526,https://product-image.wconcept.co.kr/productim...,006001-006001006-303084806_GK53446.jpg,006001006,TOM FORD,small,https://www.wconcept.co.kr/Product/303084806,Fabric 99%Cotton 1%Polyurethane Lining 100...,TOMFORD,1659000
1527,https://product-image.wconcept.co.kr/productim...,006001-006001006-303084782_KG48956.jpg,006001006,BRUNELLO CUCINELLI,small,https://www.wconcept.co.kr/Product/303084782,93%CO 7%PA,BRUNELLO CUCINELLI,1532000
1528,https://product-image.wconcept.co.kr/productim...,006001-006001006-303084774_KM99760.jpg,006001006,Loro Piana,small,https://www.wconcept.co.kr/Product/303084774,Cotone,LORO PIANA,1090000


In [16]:
df5.to_csv(f'clothes_info_바지_{code}_page{page}_20230630.csv',encoding='utf-8-sig',index=False)